In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import sys
import os
import ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings, SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/shahparkhan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import re

def extract_decision(output):
    match = re.search(r'^2\) (Don\'t Remove|Remove)$', output, re.MULTILINE)
    if match:
        return match.group(1)
    else:
        return None

In [7]:
# file_path = "original.pdf"
# loader = PyPDFLoader(file_path)
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# splits = text_splitter.split_documents(docs)

# # 2. Create Ollama embeddings and vector store
# embeddings = OllamaEmbeddings(model="llama3")
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


persist_directory = "./vector_dbs/mpnet"
# # Make New Embeddings 


# # 1. Load the text file
# file_path = "original.txt"
# loader = TextLoader(file_path)
# docs = loader.load()

# # 2. Split the document into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# splits = text_splitter.split_documents(docs)
# print(len(splits))

# # 3. Create Ollama embeddings and vector store
# embeddings = OllamaEmbeddings(model="llama3")
# embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_directory)

# Load Stored DB Embeddings
# embeddings = OllamaEmbeddings(model="llama3")
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [46]:
with open('callgraph.txt', 'r') as file:
    # Read the entire contents of the file into a string
    callgraph = file.read()

# 3. Call Ollama Llama3 model
def ollama_llm(question, context, covered, callgraph=callgraph):
    # formatted_prompt = f"Question: {question}\n\nContext: {context}"

    not_coverage_prompt = f"""You are a senior principal code architect tasked with assessing the generality and security implications of removing code snippets mentioned in
Question: {question}

The code given above does is not present in LLVM-Coverage, however, we still need to consider it for the overall generality, and security implication of removing the code.

Evaluate the impact of removing a specific code snippet. Try to understand the snippet with respect to the entire code and the given
Context: {context}

If you don't know the answer, just say that you don't know, don't try to make up an answer.
1) answer in a score from 1 to 10, that answers the importance of code to the entire program's generality and security.
2) say Remove, or Do not Remove, to tell if removing this code is fine considering the generality and security implications
3) Give an explanation on why it can or cannot be removed

Follow the exact output format shown below:
1) Importance Score : <number>
2) <Remove> or <Don't Remove> (Do not output anything else)
3) <Explanation>

Make sure you don't add any information on your own. Reply in 3 lines to cover each point.

Answer: """

    coverage_prompt = f"""You are a senior principal code architect tasked with assessing the security and functionality implications of removing code snippets mentioned in 
Question: {question}

The code given above is present in LLVM-Coverage, however, if it is not essential to the program's functionality and security, we could safely remove it.

Evaluate the impact of removing a specific code snippet. Try to understand the snippet with respect to the entire code and the given
Context: {context}

If you don't know the answer, just say that you don't know, don't try to make up an answer.
1) answer in a score from 1 to 10, that answers the importance of code to the entire program's functionality and security.
2) say Remove, or Do not Remove, to tell if removing this code is fine considering the security and functionality implications
3) Give an explanation on why it can or cannot be removed

Follow the exact output format shown below:
1) Importance Score : <number>
2) <Remove> or <Don't Remove> (Do not output anything else)
3) <Explanation>

Make sure you don't add any information on your own. Reply in 3 lines to cover each point.

Answer: """
    if covered: 
        response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': coverage_prompt}],options={"temperature":1})
    else: 
        response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': not_coverage_prompt}],options={"temperature":1})
    return response['message']['content']

# 4. RAG Setup
retriever = vectorstore.as_retriever()

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def rag_chain(question, retrieval_context, covered=False):
    retrieved_docs = retriever.invoke(retrieval_context)
    formatted_context = combine_docs(retrieved_docs)
    print("covered:",covered)
    print(f"""****
    Context Being Picked: 
{formatted_context}
****""")
    return ollama_llm(question, formatted_context, covered)



In [53]:
context = """is_dir = (_Bool)tmp___16;
    tmp___17 = prompt((FTS const *)fts, (FTSENT const *)ent, is_dir, x,
                      (enum Prompt_action)3, (Ternary *)((void *)0));
    s___0 = tmp___17;
    if ((unsigned int)s___0 != 2U) {
      return (s___0);
    }
    tmp___18 = excise(fts, ent, x, is_dir);
    return (tmp___18);
  case_2:
    while (1) {
      tmp___19 = quote((char const *)ent->fts_path);
      tmp___20 = gettext(
          "WARNING: Circular directory structure.\nThis almost certainly means "
          "that you have a corrupted file system.\nNOTIFY YOUR SYSTEM "
          "MANAGER.\nThe following directory is part of the cycle:\n  %s\n");
      error(0, 0, (char const *)tmp___20, tmp___19);
      goto while_break___0;
    }
  while_break___0:
    fts_skip_tree(fts, ent);
    return ((enum RM_status)4);
  case_7:
    tmp___21 = quote((char const *)ent->fts_path);
    tmp___22 = gettext("traversal failed: %s");
    error(0, ent->fts_errno, (char const *)tmp___22, tmp___21);
    fts_skip_tree(fts, ent);
    return ((enum RM_status)4);"""

x = rag_chain("""while (1) {
  tmp___19 = quote((char const *)ent->fts_path);
  tmp___20 = gettext(
      "WARNING: Circular directory structure.
This almost certainly means "
      "that you have a corrupted file system.
NOTIFY YOUR SYSTEM "
      "MANAGER.
The following directory is part of the cycle:
  %s
");
  error(0, 0, (char const *)tmp___20, tmp___19);
  goto while_break___0;
}""", context)

print(x)



****
    Context Being Picked: 
tmp___17 = prompt((FTS const *)fts, (FTSENT const *)ent, is_dir, x,
                      (enum Prompt_action)3, (Ternary *)((void *)0));
    s___0 = tmp___17;
    if ((unsigned int)s___0 != 2U) {
      return (s___0);
    }
    tmp___18 = excise(fts, ent, x, is_dir);
    return (tmp___18);
  case_2:
    while (1) {
      tmp___19 = quote((char const *)ent->fts_path);
      tmp___20 = gettext(
          "WARNING: Circular directory structure.\nThis almost certainly means "
          "that you have a corrupted file system.\nNOTIFY YOUR SYSTEM "
          "MANAGER.\nThe following directory is part of the cycle:\n  %s\n");
      error(0, 0, (char const *)tmp___20, tmp___19);
      goto while_break___0;
    }
  while_break___0:
    fts_skip_tree(fts, ent);
    return ((enum RM_status)4);
  case_7:
    tmp___21 = quote((char const *)ent->fts_path);
    tmp___22 = gettext("traversal failed: %s");
    error(0, ent->fts_errno, (char const *)tmp___22, tmp___21);


In [54]:
in_val = """if ((unsigned int)s == 2U) {
 if ((unsigned int)is_empty_directory == 4U) {
   s = excise(fts, ent, x, (_Bool)1);
   fts_skip_tree(fts, ent);
 }
}"""

context = """while_break:
            fts_skip_tree(fts, ent);
            return ((enum RM_status)4);
          }
        }
      }
    }
    tmp___13 = prompt((FTS const *)fts, (FTSENT const *)ent, (_Bool)1, x,
                      (enum Prompt_action)2, &is_empty_directory);
    s = tmp___13;
    if ((unsigned int)s == 2U) {
      if ((unsigned int)is_empty_directory == 4U) {
        s = excise(fts, ent, x, (_Bool)1);
        fts_skip_tree(fts, ent);
      }
    }
    if ((unsigned int)s != 2U) {
      mark_ancestor_dirs(ent);
      fts_skip_tree(fts, ent);
    }
    return (s);
  case_8:
    if ((int)ent->fts_info == 6) {
      if (x->one_file_system) {
        if (0L < ent->fts_level) {
          if (ent->fts_statp[0].st_dev != fts->fts_dev) {"""

x = rag_chain(in_val, context)

print(x)


****
    Context Being Picked: 
if ((unsigned int)s == 2U) {
      if ((unsigned int)is_empty_directory == 4U) {
        s = excise(fts, ent, x, (_Bool)1);
        fts_skip_tree(fts, ent);
      }
    }
    if ((unsigned int)s != 2U) {
      mark_ancestor_dirs(ent);
      fts_skip_tree(fts, ent);
    }
    return (s);
  case_8:
    if ((int)ent->fts_info == 6) {
      if (x->one_file_system) {
        if (0L < ent->fts_level) {
          if (ent->fts_statp[0].st_dev != fts->fts_dev) {
            mark_ancestor_dirs(ent);
            tmp___14 = quote((char const *)ent->fts_path);
            tmp___15 =
                gettext("skipping %s, since it\'s on a different device");
            error(0, 0, (char const *)tmp___15, tmp___14);
            return ((enum RM_status)4);
          }
        }
      }
    }
    if ((int)ent->fts_info == 6) {
      tmp___16 = 1;
    } else {
      if ((int)ent->fts_info == 4) {
        tmp___16 = 1;
      } else {
        tmp___16 = 0;
      }
    }
   

In [49]:
in_val = """if (status != 0) {
 tmp = gettext("Try `%s --help\' for more information.\n");
 fprintf(stderr, (char const *)tmp, program_name);
} else {
 tmp___0 = gettext("Usage: %s [OPTION]... FILE...\n");
 printf((char const *)tmp___0, program_name);
 tmp___1 = gettext("Remove (unlink) the FILE(s).\n\n  -f, --force         "
                   "  ignore nonexistent files, never prompt\n  -i        "
                   "            prompt before every removal\n");
 fputs_unlocked((char const *)tmp___1, stdout);
 tmp___2 = gettext(
     "  -I                    prompt once before removing more than three "
     "files, or\n                          when removing recursively.  "
     "Less intrusive than -i,\n                          while still "
     "giving protection against most mistakes\n      --interactive[=WHEN] "
     " prompt according to WHEN: never, once (-I), or\n                   "
     "       always (-i).  Without WHEN, prompt always\n");
 fputs_unlocked((char const *)tmp___2, stdout);
 tmp___3 =
     gettext("      --one-file-system  when removing a hierarchy "
             "recursively, skip any\n                          directory "
             "that is on a file system different from\n                   "
             "       that of the corresponding command line argument\n");
 fputs_unlocked((char const *)tmp___3, stdout);
 tmp___4 = gettext(
     "      --no-preserve-root  do not treat `/\' specially\n      "
     "--preserve-root   do not remove `/\' (default)\n  -r, -R, "
     "--recursive   remove directories and their contents recursively\n  "
     "-v, --verbose         explain what is being done\n");
 fputs_unlocked((char const *)tmp___4, stdout);
 tmp___5 = gettext("      --help     display this help and exit\n");
 fputs_unlocked((char const *)tmp___5, stdout);
 tmp___6 = gettext("      --version  output version information and exit\n");
 fputs_unlocked((char const *)tmp___6, stdout);
 tmp___7 = gettext("\nBy default, rm does not remove directories.  Use the "
                   "--recursive (-r or -R)\noption to remove each listed "
                   "directory, too, along with all of its contents.\n");
 fputs_unlocked((char const *)tmp___7, stdout);
 tmp___8 = gettext("\nTo remove a file whose name starts with a `-\', for "
                   "example `-foo\',\nuse one of these commands:\n  %s -- "
                   "-foo\n\n  %s ./-foo\n");
 printf((char const *)tmp___8, program_name, program_name);
 tmp___9 = gettext("\nNote that if you use rm to remove a file, it is "
                   "usually possible to recover\nthe contents of that "
                   "file.  If you want more assurance that the contents "
                   "are\ntruly unrecoverable, consider using shred.\n");
 fputs_unlocked((char const *)tmp___9, stdout);
 emit_ancillary_info();
}"""

context = """__attribute__((__noreturn__)) void usage(int status);
void usage(int status) {
  char *tmp;
  char *tmp___0;
  char *tmp___1;
  char *tmp___2;
  char *tmp___3;
  char *tmp___4;
  char *tmp___5;
  char *tmp___6;
  char *tmp___7;
  char *tmp___8;
  char *tmp___9;

  {
    if (status != 0) {
      tmp = gettext("Try `%s --help\' for more information.\n");
      fprintf(stderr, (char const *)tmp, program_name);
    } else {
      tmp___0 = gettext("Usage: %s [OPTION]... FILE...\n");
      printf((char const *)tmp___0, program_name);
      tmp___1 = gettext("Remove (unlink) the FILE(s).\n\n  -f, --force         "
                        "  ignore nonexistent files, never prompt\n  -i        "
                        "            prompt before every removal\n");
      fputs_unlocked((char const *)tmp___1, stdout);
      tmp___2 = gettext(
          "  -I                    prompt once before removing more than three "
          "files, or\n                          when removing recursively.  "
          "Less intrusive than -i,\n                          while still "
          "giving protection against most mistakes\n      --interactive[=WHEN] "
          " prompt according to WHEN: never, once (-I), or\n                   "
          "       always (-i).  Without WHEN, prompt always\n");
      fputs_unlocked((char const *)tmp___2, stdout);
      tmp___3 =
          gettext("      --one-file-system  when removing a hierarchy "
                  "recursively, skip any\n                          directory "
                  "that is on a file system different from\n                   "
                  "       that of the corresponding command line argument\n");
      fputs_unlocked((char const *)tmp___3, stdout);
      tmp___4 = gettext(
          "      --no-preserve-root  do not treat `/\' specially\n      "
          "--preserve-root   do not remove `/\' (default)\n  -r, -R, "
          "--recursive   remove directories and their contents recursively\n  "
          "-v, --verbose         explain what is being done\n");
      fputs_unlocked((char const *)tmp___4, stdout);
      tmp___5 = gettext("      --help     display this help and exit\n");
      fputs_unlocked((char const *)tmp___5, stdout);
      tmp___6 =
          gettext("      --version  output version information and exit\n");
      fputs_unlocked((char const *)tmp___6, stdout);
      tmp___7 =
          gettext("\nBy default, rm does not remove directories.  Use the "
                  "--recursive (-r or -R)\noption to remove each listed "
                  "directory, too, along with all of its contents.\n");
      fputs_unlocked((char const *)tmp___7, stdout);
      tmp___8 = gettext("\nTo remove a file whose name starts with a `-\', for "
                        "example `-foo\',\nuse one of these commands:\n  %s -- "
                        "-foo\n\n  %s ./-foo\n");
      printf((char const *)tmp___8, program_name, program_name);
      tmp___9 = gettext("\nNote that if you use rm to remove a file, it is "
                        "usually possible to recover\nthe contents of that "
                        "file.  If you want more assurance that the contents "
                        "are\ntruly unrecoverable, consider using shred.\n");
      fputs_unlocked((char const *)tmp___9, stdout);
      emit_ancillary_info();
    }
    exit(status);
  }
}
static void rm_option_init(struct rm_options *x) {

  {
    x->ignore_missing_files = (_Bool)0;
    x->interactive = (enum rm_interactive)4;
    x->one_file_system = (_Bool)0;"""

x = rag_chain(in_val, context, False)

print(x)


covered: False
****
    Context Being Picked: 
{
    if (status != 0) {
      tmp = gettext("Try `%s --help\' for more information.\n");
      fprintf(stderr, (char const *)tmp, program_name);
    } else {
      tmp___0 = gettext("Usage: %s [OPTION]... FILE...\n");
      printf((char const *)tmp___0, program_name);
      tmp___1 = gettext("Remove (unlink) the FILE(s).\n\n  -f, --force         "
                        "  ignore nonexistent files, never prompt\n  -i        "
                        "            prompt before every removal\n");
      fputs_unlocked((char const *)tmp___1, stdout);
      tmp___2 = gettext(
          "  -I                    prompt once before removing more than three "
          "files, or\n                          when removing recursively.  "
          "Less intrusive than -i,\n                          while still "
          "giving protection against most mistakes\n      --interactive[=WHEN] "
          " prompt according to WHEN: never, once (-I), o

In [50]:
in_val = """static struct hash_entry *allocate_entry(Hash_table *table___0) {
 struct hash_entry *new;


 {
   if (table___0->free_entry_list) {
     new = table___0->free_entry_list;
     table___0->free_entry_list = new->next;
   } else {
     new = (struct hash_entry *)malloc(sizeof(*new));
   }
   return (new);
 }
}"""

context ="""next = cursor->next;
      free((void *)cursor);
      cursor = next;
    }
  while_break___3:
    free((void *)table___0->bucket);
    free((void *)table___0);
    return;
  }
}
static struct hash_entry *allocate_entry(Hash_table *table___0) {
  struct hash_entry *new;

  {
    if (table___0->free_entry_list) {
      new = table___0->free_entry_list;
      table___0->free_entry_list = new->next;
    } else {
      new = (struct hash_entry *)malloc(sizeof(*new));
    }
    return (new);
  }
}
static void free_entry(Hash_table *table___0, struct hash_entry *entry) {

  {
    entry->data = (void *)0;
    entry->next = table___0->free_entry_list;
    table___0->free_entry_list = entry;
    return;
  }
}
static void *hash_find_entry(Hash_table *table___0, void const *entry,"""

x = rag_chain(in_val, context, True)

print(x)


covered: True
****
    Context Being Picked: 
if (!cursor) {
          goto while_break___2;
        }
        next = cursor->next;
        free((void *)cursor);
        cursor = next;
      }
    while_break___2:
      bucket++;
    }
  while_break___1:
    cursor = table___0->free_entry_list;
    while (1) {

      if (!cursor) {
        goto while_break___3;
      }
      next = cursor->next;
      free((void *)cursor);
      cursor = next;
    }
  while_break___3:
    free((void *)table___0->bucket);
    free((void *)table___0);
    return;
  }
}
static struct hash_entry *allocate_entry(Hash_table *table___0) {
  struct hash_entry *new;

  {
    if (table___0->free_entry_list) {
      new = table___0->free_entry_list;
      table___0->free_entry_list = new->next;
    } else {
      new = (struct hash_entry *)malloc(sizeof(*new));
    }
    return (new);
  }
}
static void free_entry(Hash_table *table___0, struct hash_entry *entry) {

if (!cursor) {
          goto while_break___2;
  

In [57]:
in_val = """tmp___16 = 0;"""

context = """}
        }
      }
    }
    if ((int)ent->fts_info == 6) {
      tmp___16 = 1;
    } else {
      if ((int)ent->fts_info == 4) {
        tmp___16 = 1;
      } else {
        tmp___16 = 0;
      }
    }
    is_dir = (_Bool)tmp___16;
    tmp___17 = prompt((FTS const *)fts, (FTSENT const *)ent, is_dir, x,
                      (enum Prompt_action)3, (Ternary *)((void *)0));
    s___0 = tmp___17;
    if ((unsigned int)s___0 != 2U) {
      return (s___0);
    }
    tmp___18 = excise(fts, ent, x, is_dir);"""

x = rag_chain(in_val, context)

print(x)


****
    Context Being Picked: 
skip_stat = (_Bool)tmp___17;
          p->fts_info = (unsigned short)11;
          set_stat_type(p->fts_statp, (unsigned int)dp->d_type);
          fts_set_stat_required(p, (_Bool)(!skip_stat));
          if (sp->fts_options & 16) {
            if ((int)dp->d_type == 4) {
              tmp___18 = 1;
            } else {
              tmp___18 = 0;
            }
          } else {
            tmp___18 = 0;
          }
          is_dir = (_Bool)tmp___18;
        } else {
          p->fts_info = fts_stat(sp, p, (_Bool)0);
          if ((int)p->fts_info == 1) {
            tmp___19 = 1;
          } else {
            if ((int)p->fts_info == 2) {
              tmp___19 = 1;
            } else {
              if ((int)p->fts_info == 5) {
                tmp___19 = 1;
              } else {
                tmp___19 = 0;
              }
            }
          }
          is_dir = (_Bool)tmp___19;
        }
      }
      if (nlinks > 0UL) {
        if (is_dir) {

In [51]:
in_val = """tmp = 512;"""

context = """int *tmp___7;
  int *tmp___8;
  int *tmp___9;
  _Bool tmp___10;
  int *tmp___11;
  char const *tmp___12;
  char *tmp___13;
  int *tmp___14;

  {
    if (is_dir) {
      tmp = 512;
    } else {
      tmp = 0;
    }
    flag = tmp;
    tmp___4 = unlinkat(fts->fts_cwd_fd, (char const *)ent->fts_accpath, flag);
    if (tmp___4 == 0) {
      if (x->verbose) {
        tmp___0 = quote((char const *)ent->fts_path);
        if (is_dir) {
          tmp___1 = gettext("removed directory: %s\n");"""

x = rag_chain(in_val, context, True)

print(x)


covered: True
****
    Context Being Picked: 
{
    if (is_dir) {
      tmp = 512;
    } else {
      tmp = 0;
    }
    flag = tmp;
    tmp___4 = unlinkat(fts->fts_cwd_fd, (char const *)ent->fts_accpath, flag);
    if (tmp___4 == 0) {
      if (x->verbose) {
        tmp___0 = quote((char const *)ent->fts_path);
        if (is_dir) {
          tmp___1 = gettext("removed directory: %s\n");
          tmp___3 = tmp___1;
        } else {
          tmp___2 = gettext("removed %s\n");
          tmp___3 = tmp___2;
        }
        printf((char const *)tmp___3, tmp___0);
      }
      return ((enum RM_status)2);
    }
    tmp___8 = __errno_location();
    if (*tmp___8 == 30) {
      tmp___6 = lstatat(fts->fts_cwd_fd, (char const *)ent->fts_accpath, &st);
      if (tmp___6) {
        tmp___7 = __errno_location();
        if (!(*tmp___7 == 2)) {
          tmp___5 = __errno_location();
          *tmp___5 = 30;
        }
      } else {
        tmp___5 = __errno_location();
        *tmp___5 = 30;
 

In [59]:
in_val = """char const diacrit_diac[256] = {
(char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
(char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
(char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
(char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
"""

context = """(char const)'U', (char const)'Y', (char const)0, (char const)0,
    (char const)'a', (char const)'a', (char const)'a', (char const)'a',
    (char const)'a', (char const)'a', (char const)'a', (char const)'c',
    (char const)'e', (char const)'e', (char const)'e', (char const)'e',
    (char const)'i', (char const)'i', (char const)'i', (char const)'i',
    (char const)0, (char const)'n', (char const)'o', (char const)'o',
    (char const)'o', (char const)'o', (char const)'o', (char const)0,
    (char const)'o', (char const)'u', (char const)'u', (char const)'u',
    (char const)'u', (char const)'y', (char const)0, (char const)'y'};
char const diacrit_diac[256] = {
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0, (char const)0,
   """

x = rag_chain(in_val, context)

print(x)


****
    Context Being Picked: 
(char const)'X', (char const)'Y', (char const)'Z', (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)'a', (char const)'b', (char const)'c',
    (char const)'d', (char const)'e', (char const)'f', (char const)'g',
    (char const)'h', (char const)'i', (char const)'j', (char const)'k',
    (char const)'l', (char const)'m', (char const)'n', (char const)'o',
    (char const)'p', (char const)'q', (char const)'r', (char const)'s',
    (char const)'t', (char const)'u', (char const)'v', (char const)'w',
    (char const)'x', (char const)'y', (char const)'z', (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0,
    (char const)0, (char const)0, (char const)0, (char const)0,

(char const)'X', 

In [60]:
in_val = """extern int fprintf(FILE *__restrict __stream, char const *__restrict __format,
                   ...);"""

context = """extern int(__attribute__((__nonnull__(1))) closedir)(DIR *__dirp);
extern struct dirent *(__attribute__((__nonnull__(1))) readdir)(DIR *__dirp);
DIR *opendir_safer(char const *name);
char const *simple_backup_suffix = "~";
ptrdiff_t argmatch(char const *arg, char const *const *arglist,
                   char const *vallist, size_t valsize);
void argmatch_invalid(char const *context, char const *value,
                      ptrdiff_t problem);
void argmatch_valid(char const *const *arglist, char const *vallist,
                    size_t valsize);
extern int fprintf(FILE *__restrict __stream, char const *__restrict __format,
                   ...);
extern int putc_unlocked(int __c, FILE *__stream);
char *quotearg_n_style(int n, enum quoting_style s, char const *arg);
char const *quote_n(int n, char const *name);
__attribute__((__noreturn__)) void usage(int status);
static void __argmatch_die(void) {

  {
    usage(1);
    return;
  }"""

x = rag_chain(in_val, context)

print(x)


****
    Context Being Picked: 
if (1UL < len) {
        if (!((int const) * (name + (len - 1UL)) == 47)) {
          goto while_break;
        }
      } else {
        goto while_break;
      }
      goto __Cont;
    __Cont:
      len--;
    }
  while_break:;
    return (len);
  }
}
char const *simple_backup_suffix;
void (*argmatch_die)(void);
ptrdiff_t __xargmatch_internal(char const *context, char const *arg,
                               char const *const *arglist, char const *vallist,
                               size_t valsize, void (*exit_fn)(void));
extern __attribute__((__nothrow__)) int(
    __attribute__((__nonnull__(1, 2), __leaf__))
    memcmp)(void const *__s1, void const *__s2, size_t __n)
    __attribute__((__pure__));
extern int(__attribute__((__nonnull__(1))) closedir)(DIR *__dirp);
extern struct dirent *(__attribute__((__nonnull__(1))) readdir)(DIR *__dirp);
DIR *opendir_safer(char const *name);
char const *simple_backup_suffix = "~";

if (1UL < len) {
        if 

In [61]:
in_val = "p->fts_accpath = tmp___0;"

context = """if (*(cp + 1)) {
          cp++;
          len = strlen((char const *)cp);
          memmove((void *)(p->fts_name), (void const *)cp, len + 1UL);
          p->fts_namelen = len;
        }
      }
    }
    tmp___0 = sp->fts_path;
    p->fts_path = tmp___0;
    p->fts_accpath = tmp___0;
    return;
  }
}
__attribute__((__nothrow__)) int(__attribute__((__warn_unused_result__,
                                                __leaf__)) fts_close)(FTS *sp);
int(__attribute__((__warn_unused_result__, __leaf__)) fts_close)(FTS *sp) {
  register FTSENT *freep;
  register FTSENT *p;
  int saved_errno;
  int *tmp;"""

x = rag_chain(in_val, context)

print(x)


****
    Context Being Picked: 
{
    tmp = p->fts_namelen;
    p->fts_pathlen = tmp;
    len = tmp;
    memmove((void *)sp->fts_path, (void const *)(p->fts_name), len + 1UL);
    cp = strrchr((char const *)(p->fts_name), '/');
    if (cp) {
      if ((unsigned long)cp != (unsigned long)(p->fts_name)) {
        cp++;
        len = strlen((char const *)cp);
        memmove((void *)(p->fts_name), (void const *)cp, len + 1UL);
        p->fts_namelen = len;
      } else {
        if (*(cp + 1)) {
          cp++;
          len = strlen((char const *)cp);
          memmove((void *)(p->fts_name), (void const *)cp, len + 1UL);
          p->fts_namelen = len;
        }
      }
    }
    tmp___0 = sp->fts_path;
    p->fts_path = tmp___0;
    p->fts_accpath = tmp___0;
    return;
  }
}
__attribute__((__nothrow__)) int(__attribute__((__warn_unused_result__,
                                                __leaf__)) fts_close)(FTS *sp);
int(__attribute__((__warn_unused_result__, __leaf__)) fts_clos

In [42]:
in_val = """static void rm_option_init(struct rm_options *x)
{

  {
    x->ignore_missing_files = (_Bool)0;
    x->interactive = (enum rm_interactive)4;
    x->one_file_system = (_Bool)0;
    x->recursive = (_Bool)0;
    x->root_dev_ino = (struct dev_ino *)((void *)0);
    x->stdin_tty = (_Bool)isatty(0);
    x->verbose = (_Bool)0;
    x->require_restore_cwd = (_Bool)0;
    return;
  }
}"""

context = """fputs_unlocked((char const *)tmp___7, stdout);
      tmp___8 = gettext("\nTo remove a file whose name starts with a `-\', for "
                        "example `-foo\',\nuse one of these commands:\n  %s -- "
                        "-foo\n\n  %s ./-foo\n");
      printf((char const *)tmp___8, program_name, program_name);
      tmp___9 = gettext("\nNote that if you use rm to remove a file, it is "
                        "usually possible to recover\nthe contents of that "
                        "file.  If you want more assurance that the contents "
                        "are\ntruly unrecoverable, consider using shred.\n");
      fputs_unlocked((char const *)tmp___9, stdout);
      emit_ancillary_info();
    }
    exit(status);
  }
}
static void rm_option_init(struct rm_options *x)
{

  {
    x->ignore_missing_files = (_Bool)0;
    x->interactive = (enum rm_interactive)4;
    x->one_file_system = (_Bool)0;
    x->recursive = (_Bool)0;
    x->root_dev_ino = (struct dev_ino *)((void *)0);
    x->stdin_tty = (_Bool)isatty(0);
    x->verbose = (_Bool)0;
    x->require_restore_cwd = (_Bool)0;
    return;
  }
}
static struct dev_ino dev_ino_buf;
int main(int argc, char **argv)
{
  _Bool preserve_root;
  struct rm_options x;
  _Bool prompt_once;
  int c;
  int i;
  ptrdiff_t tmp;
  char *tmp___0;
  char const *tmp___1;
  char *tmp___2;
  int *tmp___3;
  size_t n_files;
  char **file;"""

x = rag_chain(in_val, context, True)

print(x)


covered: True
****
    Context Being Picked: 
tmp___6 =
          gettext("      --version  output version information and exit\n");
      fputs_unlocked((char const *)tmp___6, stdout);
      tmp___7 =
          gettext("\nBy default, rm does not remove directories.  Use the "
                  "--recursive (-r or -R)\noption to remove each listed "
                  "directory, too, along with all of its contents.\n");
      fputs_unlocked((char const *)tmp___7, stdout);
      tmp___8 = gettext("\nTo remove a file whose name starts with a `-\', for "
                        "example `-foo\',\nuse one of these commands:\n  %s -- "
                        "-foo\n\n  %s ./-foo\n");
      printf((char const *)tmp___8, program_name, program_name);
      tmp___9 = gettext("\nNote that if you use rm to remove a file, it is "
                        "usually possible to recover\nthe contents of that "
                        "file.  If you want more assurance that the contents "

tmp___6 =
  